# CGC PSU with Base Class Testing

This notebook tests the new PSU class structure with PSUBase and PSU (with logging).

## Test Sections:
1. **Internal Logger**: PSU creates its own logger automatically
2. **External Logger**: Using a custom external logger

In [1]:
# Import required modules
import sys
import os
import logging

# Add src to path
sys.path.append(os.path.join(os.getcwd(), '..', '..', 'src'))

from devices.cgc.psu.psu import PSU

## Section 1: Internal Logger Testing

Test PSU with its own internal logger (automatic file creation).

In [2]:
# Create PSU instance with internal logger
psu_internal = PSU("test_psu_internal", com=6, port=1)

print("PSU created with internal logger")
print(f"Device ID: {psu_internal.device_id}")
print(f"Logger: {psu_internal.logger.name}")

PSU created with internal logger
Device ID: test_psu_internal
Logger: PSU_test_psu_internal_20251105_115512


In [3]:
# Connect to device (will be logged)
print("Attempting to connect...")
result = psu_internal.connect()
print(f"Connection result: {result}")


Attempting to connect...
Connection result: True


In [4]:
psu_internal._hk_product_info()

True

In [23]:
psu_internal.get_sensor_data()

(0, 19.54, 19.06, 18.94)

In [17]:
psu_internal._hk_main_state()

True

In [6]:
psu_internal.logger.error("HELLO")

In [26]:
psu_internal.logger

<Logger PSU_test_psu_internal_20251105_112505 (INFO)>

In [16]:
# Test some function calls with automatic logging via __getattr__
print("\nTesting method calls with automatic logging:")

# Test get_main_state (should be logged automatically)
try:
    status, state_hex, state_name = psu_internal.get_main_state()
    print(status, state_hex, state_name)
except Exception as e:
    print(f"get_main_state error: {e}")


Testing method calls with automatic logging:
0 0x8004 STATE_ERR_ILOCK


In [18]:
# Test get_device_enable (should be logged automatically)
try:
    status, enabled = psu_internal.get_device_enable()
    print(f"Device enabled: {enabled}")
except Exception as e:
    print(f"get_device_enable error: {e}")


Device enabled: False


In [7]:
psu_internal.disconnect()

True

## Section 2: External Logger Testing

Test PSU with a custom external logger.

In [ ]:
# Create external logger
from datetime import datetime
from pathlib import Path

# Setup external logger
external_logger = logging.getLogger("ExternalPSULogger")
external_logger.setLevel(logging.DEBUG)

# Create file handler for external logger
logs_dir = Path("../logs")
logs_dir.mkdir(exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_file = logs_dir / f"PSU_external_{timestamp}.log"

file_handler = logging.FileHandler(log_file)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
file_handler.setFormatter(formatter)
external_logger.addHandler(file_handler)

print(f"External logger created: {log_file}")
print(f"Logger level: {external_logger.level}")
print(f"Logger handlers: {len(external_logger.handlers)}")

In [ ]:
# Create PSU instance with external logger
psu_external = PSU("test_psu_external", com=6, port=1, logger=external_logger)

print("PSU created with external logger")
print(f"Device ID: {psu_external.device_id}")
print(f"Using external logger: {psu_external._external_logger_provided}")

# Connect to device
print("\nAttempting to connect...")
result = psu_external.connect()
print(f"Connection result: {result}")

# Test function calls with external logging
print("\nTesting method calls with external logger:")

# Test get_housekeeping (automatic logging via __getattr__)
try:
    status, volt_rect, volt_5v0, volt_3v3, temp_cpu = psu_external.get_housekeeping()
    print(f"Housekeeping - Voltage rect: {volt_rect:.2f}V, CPU temp: {temp_cpu:.1f}°C")
except Exception as e:
    print(f"get_housekeeping error: {e}")

# Test get_psu0_output_voltage (explicitly overridden method)
try:
    status, voltage = psu_external.get_psu0_output_voltage()
    print(f"PSU0 output voltage: {voltage:.3f}V")
except Exception as e:
    print(f"get_psu0_output_voltage error: {e}")

print(f"\nCheck the log file: {log_file}")
print("Both automatic (__getattr__) and explicit method logging should be visible")